#  Description

This experiment, as highlighted in the interim report, will randomly generate 500 linear problems and test my solver's solution against a public solver solution.
The LPs at hand will have sizes: 5x5, 10x10, 15x15, 20x20, 25x25, having 100 problems from each category.



# Generate the 500 random problems. 

In [2]:
from dense_lp_generator import DenseLPGenerator
import random
import os

# IMPORTANT: Set random seed for reproducibility.
random.seed(42)

sizes = [(i, i) for i in range(5, 26, 5)]
root_folder = './problems/problems_correctness'

if not os.path.exists(root_folder):
    os.mkdir(root_folder)

# Consider a level of precision of 3 decimals.  
lp_generator = DenseLPGenerator(3)

## The next block of code will generate the problems, i.e. create the files. It will only be run once.

In [ ]:
# for (num_variables, num_constraints) in sizes:
#     problems_folder = os.path.join(root_folder, f'{num_variables}x{num_constraints}')
#     if not os.path.exists(problems_folder):
#         os.mkdir(problems_folder)

#     for i in range(100):
#         lp_generator.generate_dense_lp(os.path.join(problems_folder, f'{i+1}.lp'), num_variables, num_constraints)
      

# Generate solutions with GLPK.

In [1]:
import subprocess
import re
import math

def solve_lp_with_glpk(lp_filepath):
    """
    Solves the given problem using GLPK and returns the status as well as the optimal solution (if there is one).
    """
    command = ["glpsol", "--lp", lp_filepath]
    result = subprocess.run(command, text=True, stdout=subprocess.PIPE)

    # Check for specific messages in the output
    output = result.stdout
    if "OPTIMAL LP SOLUTION FOUND" in output:
        # Extract the optimal objective value
        matches = re.findall(r"\*\s+\d+:\s+obj\s+=\s+([-\d.e+]+)", output)
        if matches:
            # Convert the last match to a float and return
            optimal_value = float(matches[-1])
            return {"status": "Optimal", "value": optimal_value}
    elif "LP HAS UNBOUNDED PRIMAL SOLUTION" in output:
        return {"status": "Unbounded", "value": math.inf}
    elif "PROBLEM HAS NO FEASIBLE SOLUTION" in output:
        return {"status": "Infeasible", "value": None}
    
    return {"status": "Error", "value": None}
  


In [6]:
glpk_results = {f'{i[0]}x{i[1]}': {} for i in sizes}
for (num_variables, num_constraints) in sizes:
    problems_folder = os.path.join(root_folder, f'{num_variables}x{num_constraints}') 
    for i in range(1, 101):
        glpk_output = solve_lp_with_glpk(os.path.join(problems_folder, f'{i}.lp'))
        glpk_results[f'{num_variables}x{num_constraints}'][f'{i}.lp'] = glpk_output

In [7]:
import json
with open(os.path.join(root_folder, 'glpk_results.json'), 'w') as f:
    json.dump(glpk_results, f, indent=4)

# Generate solutions with my own solver.

In [3]:
from simplex_solver import SimplexSolver
from input_parser import LPParser

my_solver = SimplexSolver(pivot_rule="Dantzig")
lp_parser = LPParser()

In [9]:
my_solver_results = {f'{i[0]}x{i[1]}': {} for i in sizes}
for (num_variables, num_constraints) in sizes:
    problems_folder = os.path.join(root_folder, f'{num_variables}x{num_constraints}') 
    for i in range(1, 101):
        lp_parser.parse_file(os.path.join(problems_folder, f'{i}.lp'))
        my_solver_output = my_solver.solve(lp_parser)
        my_solver_results[f'{num_variables}x{num_constraints}'][f'{i}.lp'] = my_solver_output

In [10]:
import json
with open(os.path.join(root_folder, 'my_solver_results.json'), 'w') as f:
    json.dump(my_solver_results, f, indent=4)

# Comparing both solvers' solutions.

In [11]:
with open(os.path.join(root_folder, 'my_solver_results.json'), 'r') as f:
    my_solver_results = json.load(f)

with open(os.path.join(root_folder, 'glpk_results.json'), 'r') as f:
    glpk_results = json.load(f)

In [12]:
for (num_variables, num_constraints) in sizes:
    my_solver_temp = my_solver_results[f'{num_variables}x{num_constraints}']
    glpk_temp = glpk_results[f'{num_variables}x{num_constraints}']

    for i in range(1, 41):
        my_solver_value = my_solver_temp[f'{i}.lp']['value']
        glpk_value = glpk_temp[f'{i}.lp']['value']
        if glpk_value is None:
            continue
        
        assert round(my_solver_value, 4) == round(glpk_value, 4)


# ALL ASSERTIONS PASSED!! (APART FROM 16 PROBLEMS WHICH GLPK CANNOT SOLVE)

### Now I want to test Steepest Edge rule, check if it's correct.

In [5]:
import json

steepest_edge_solver = SimplexSolver(pivot_rule="SteepestEdge")

In [6]:
my_solver_results = {f'{i[0]}x{i[1]}': {} for i in sizes}
for (num_variables, num_constraints) in sizes:
    problems_folder = os.path.join(root_folder, f'{num_variables}x{num_constraints}') 
    for i in range(1, 101):
        lp_parser.parse_file(os.path.join(problems_folder, f'{i}.lp'))
        my_solver_output = steepest_edge_solver.solve(lp_parser)
        my_solver_results[f'{num_variables}x{num_constraints}'][f'{i}.lp'] = my_solver_output

In [7]:
with open(os.path.join(root_folder, 'steepest_edge_results.json'), 'w') as f:
    json.dump(my_solver_results, f, indent=4)

In [11]:
with open(os.path.join(root_folder, 'my_solver_results.json'), 'r') as f:
    my_solver_results = json.load(f)

with open(os.path.join(root_folder, 'steepest_edge_results.json'), 'r') as f:
    steepest_edge_results = json.load(f)

In [ ]:
for (num_variables, num_constraints) in sizes:
    my_solver_temp = my_solver_results[f'{num_variables}x{num_constraints}']
    steepest_temp = steepest_edge_results[f'{num_variables}x{num_constraints}']

    for i in range(1, 41):
        my_solver_value = my_solver_temp[f'{i}.lp']['value']
        steepest_value = steepest_temp[f'{i}.lp']['value']
        if steepest_value is None:
            continue
        
        assert round(my_solver_value, 4) == round(steepest_value, 4)

print("Steepest Edge Works As Intended")
